In [2]:
from __future__ import absolute_import, division, print_function
import torch
from torch.distributions import constraints
from torch.nn import Parameter
import pyro
from pyro.contrib.gp.util import Parameterized
import pyro.distributions as dist
import pyro.infer as infer
import pyro.optim as optim
from pyro.params import param_with_module_name
import pyro.contrib.gp as gp
import pyro.distributions as dist
import matplotlib.pyplot as plt
pyro.clear_param_store()
import tensorflow as tf
import numpy as np 
import pandas as pd
import numpy
import math
import cProfile
import time
torch.manual_seed(101)
pyro.set_rng_seed(1)
import GPy
import numpy as np
from matplotlib import pyplot as plt 

In [3]:
y = torch.stack([dist.Normal(4.8, 0.1).sample((150,)),dist.Normal(3.2, 0.3).sample((150,)),dist.Normal(1.5, 0.4).sample((150,)),
 dist.Exponential(0.5).sample((150,))])
X_loc = torch.zeros(150, 2)
kernel = gp.kernels.RBF(input_dim=2,lengthscale=torch.ones(1))
Xu = torch.zeros(20, 2)
gpmodel = gp.models.SparseGPRegression(X_loc, y, kernel, Xu, noise=torch.tensor(1.))
gplvm = gp.models.GPLVM(gpmodel)

In [4]:
losses = gplvm.optimize(num_steps=2000)

In [5]:
X = gplvm.get_param("X_loc")
X = X.detach().numpy()
X_var = gplvm.get_param("X_scale_tril")
X_var = X_var.detach().numpy()

### Sampling from a variational distribution

In [6]:
Samples=150
VS = []
for i in range(Samples):
    VS.append(pyro.sample('Variational Distibtuion',dist.MultivariateNormal(gplvm.get_param("X_loc"), scale_tril=gplvm.get_param("X_scale_tril")).independent(gplvm.get_param("X_loc").dim()-1),))


## Bayesian GPLVM

In [8]:
Y = numpy.array(y)
Y = Y.T
kernel = GPy.kern.RBF(2, ARD=True,variance=1.0,lengthscale=1.0)

In [9]:
m = GPy.models.BayesianGPLVM(Y, 2,  init="PCA",kernel=kernel,num_inducing=20)
m.data_labels = Y.argmax(axis=1)
loss = m.optimize('scg', messages=1,max_iters=2000)

Running Scaled Conjugate Gradients Code:
  runtime   i      f              |g|        
    00s16  0004   1.213564e+03   4.125049e+04 
    02s21  0065   5.542470e+02   4.418814e+03 
    03s23  0095   5.448641e+02   2.012194e+02 
    10s36  0324   4.654211e+02   3.912429e+01 

 /home/aneeqr/.local/lib/python3.6/site-packages/GPy/kern/src/stationary.py:165: RuntimeWarning:overflow encountered in true_divide
 /home/aneeqr/.local/lib/python3.6/site-packages/GPy/kern/src/stationary.py:137: RuntimeWarning:invalid value encountered in add



    15s45  0445   4.618513e+02   2.896723e+01 

 /home/aneeqr/.local/lib/python3.6/site-packages/GPy/kern/src/stationary.py:136: RuntimeWarning:overflow encountered in square


    31s73  0962   4.408334e+02   2.026809e+01 
    47s58  1455   4.046699e+02   1.885049e+01 
Runtime:     47s58
Optimization status: converged - relative stepsize



## Settting up the Covariance Matrix for Joint PDF

We evaluate Joint PDF as follows

$$
p(\mathbf{y}|\mathbf{X}) = \frac{1}{(2\pi)^{\frac{n}{2}}|\mathbf{K}|^{\frac{1}{2}}} \exp\left(-\frac{1}{2}\mathbf{y}^\top \left(\mathbf{K}+\sigma^2 \mathbf{I}\right)^{-1}\mathbf{y}\right)
$$

In [13]:
def Joint_Prob(y,X,kernel,D):
    K= kernel.K(np.array(X),np.array(X)) # Covariance matrix for X prior
    noise_var = 0.05
    Noise=np.eye(K.shape[0])*np.sqrt(noise_var) 
    Noise_term = np.matmul(np.linalg.inv(Noise),np.eye(K.shape[0]))  # Noise term
    K_new = np.add(K,Noise_term) #  Covariance matrix for
    p_yx=1
    for i in range(D):
        p_yx = p_yx*(1/(np.sqrt(2*np.pi)**(int(K_new.shape[0]/2))*np.linalg.det(K_new)))*np.exp(-1/2*np.matmul(np.matmul(y[:,i].reshape(K_new.shape[0],1).T,np.linalg.inv(K_new)),y[:,i].reshape(K_new.shape[0],1)))
    return p_yx

We evaluate variational distribution as follows:

$$
q(\mathbf{X}) = \frac{1}{(2\pi)^{\frac{n}{2}}|\mathbf{K}|^{\frac{1}{2}}} \exp\left(-\frac{1}{2}\mathbf{x}^\top \left(\mathbf{K}+\sigma^2 \mathbf{I}\right)^{-1}\mathbf{x}\right)
$$

In [16]:
def Variational_Dist(X,D,kernel):
    qx = 1
    K= kernel.K(np.array(X),np.array(X)) # Covariance matrix for X prior
    noise_var = 0.05
    Noise=np.eye(K.shape[0])*np.sqrt(noise_var) 
    Noise_term = np.matmul(np.linalg.inv(Noise),np.eye(K.shape[0]))  # Noise term
    K_new = np.add(K,Noise_term) #  Covariance matrix for
    qx=1
    for i in range(D):
        qx = qx*(1/(np.sqrt(2*np.pi)**(int(K_new.shape[0]/2))*np.linalg.det(K_new)))*np.exp(-1/2*np.matmul(np.matmul(X[:,i].reshape(K_new.shape[0],1).T,np.linalg.inv(K_new)),X[:,i].reshape(K_new.shape[0],1)))
    return qx  

# PSIS Functions

In [18]:
def psislw(lw, Reff=1.0, overwrite_lw=False):
    """Pareto smoothed importance sampling (PSIS).

    Parameters
    ----------
    lw : ndarray
        Array of size n x m containing m sets of n log weights. It is also
        possible to provide one dimensional array of length n.

    Reff : scalar, optional
        relative MCMC efficiency ``N_eff / N``

    overwrite_lw : bool, optional
        If True, the input array `lw` is smoothed in-place, assuming the array
        is F-contiguous. By default, a new array is allocated.

    Returns
    -------
    lw_out : ndarray
        smoothed log weights
    kss : ndarray
        Pareto tail indices

    """
    if lw.ndim == 2:
        n, m = lw.shape
    elif lw.ndim == 1:
        n = len(lw)
        m = 1
    else:
        raise ValueError("Argument `lw` must be 1 or 2 dimensional.")
    if n <= 1:
        raise ValueError("More than one log-weight needed.")

    if overwrite_lw and lw.flags.f_contiguous:
        # in-place operation
        lw_out = lw
    else:
        # allocate new array for output
        lw_out = np.copy(lw, order='F')

    # allocate output array for kss
    kss = np.empty(m)
    #print(kss)

    # precalculate constants
    cutoff_ind = - int(np.ceil(min(0.2 * n, 3 * np.sqrt(n / Reff)))) - 1
    cutoffmin = np.log(np.finfo(float).tiny)
    logn = np.log(n)
    k_min = 1/3

    # loop over sets of log weights
    for i, x in enumerate(lw_out.T if lw_out.ndim == 2 else lw_out[None, :]):
        # improve numerical accuracy
        x -= np.max(x)
        # sort the array
        x_sort_ind = np.argsort(x)
        # divide log weights into body and right tail
        xcutoff = max(
            x[x_sort_ind[cutoff_ind]],
            cutoffmin
        )
        expxcutoff = np.exp(xcutoff)
        tailinds, = np.where(x > xcutoff)
        x2 = x[tailinds]
        n2 = len(x2)
        if n2 <= 4:
            # not enough tail samples for gpdfitnew
            k = np.inf
        else:
            # order of tail samples
            x2si = np.argsort(x2)
            # fit generalized Pareto distribution to the right tail samples
            np.exp(x2, out=x2)
            x2 -= expxcutoff
            k, sigma = gpdfitnew(x2, sort=x2si)
        if k >= k_min and not np.isinf(k):
            # no smoothing if short tail or GPD fit failed
            # compute ordered statistic for the fit
            sti = np.arange(0.5, n2)
            sti /= n2
            qq = gpinv(sti, k, sigma)
            qq += expxcutoff
            np.log(qq, out=qq)
            # place the smoothed tail into the output array
            x[tailinds[x2si]] = qq
            # truncate smoothed values to the largest raw weight 0
            x[x > 0] = 0
        # renormalize weights
        x -= sumlogs(x)
        # store tail index k
        kss[i] = k
        #print(k)

    # If the provided input array is one dimensional, return kss as scalar.
    if lw_out.ndim == 1:
        kss = kss[0]

    return lw_out, kss

def sumlogs(x, axis=None, out=None):
    """Sum of vector where numbers are represented by their logarithms.

    Calculates ``np.log(np.sum(np.exp(x), axis=axis))`` in such a fashion that
    it works even when elements have large magnitude.

    """
    maxx = x.max(axis=axis, keepdims=True)
    xnorm = x - maxx
    np.exp(xnorm, out=xnorm)
    out = np.sum(xnorm, axis=axis, out=out)
    if isinstance(out, np.ndarray):
        np.log(out, out=out)
    else:
        out = np.log(out)
    out += np.squeeze(maxx)
    return out
def gpdfitnew(x, sort=True, sort_in_place=False, return_quadrature=False):
    """Estimate the paramaters for the Generalized Pareto Distribution (GPD)

    Returns empirical Bayes estimate for the parameters of the two-parameter
    generalized Parato distribution given the data.

    Parameters
    ----------
    x : ndarray
        One dimensional data array

    sort : bool or ndarray, optional
        If known in advance, one can provide an array of indices that would
        sort the input array `x`. If the input array is already sorted, provide
        False. If True (default behaviour), the array is sorted internally.

    sort_in_place : bool, optional
        If `sort` is True and `sort_in_place` is True, the array is sorted
        in-place (False by default).

    return_quadrature : bool, optional
        If True, quadrature points and weight `ks` and `w` of the marginal posterior distribution of k are also calculated and returned. False by
        default.

    Returns
    -------
    k, sigma : float
        estimated parameter values

    ks, w : ndarray
        Quadrature points and weights of the marginal posterior distribution
        of `k`. Returned only if `return_quadrature` is True.

    Notes
    -----
    This function returns a negative of Zhang and Stephens's k, because it is
    more common parameterisation.

    """
    if x.ndim != 1 or len(x) <= 1:
        raise ValueError("Invalid input array.")

    # check if x should be sorted
    if sort is True:
        if sort_in_place:
            x.sort()
            xsorted = True
        else:
            sort = np.argsort(x)
            xsorted = False
    elif sort is False:
        xsorted = True
    else:
        xsorted = False

    n = len(x)
    PRIOR = 3
    m = 30 + int(np.sqrt(n))

    bs = np.arange(1, m + 1, dtype=float)
    bs -= 0.5
    np.divide(m, bs, out=bs)
    np.sqrt(bs, out=bs)
    np.subtract(1, bs, out=bs)
    if xsorted:
        bs /= PRIOR * x[int(n/4 + 0.5) - 1]
        bs += 1 / x[-1]
    else:
        bs /= PRIOR * x[sort[int(n/4 + 0.5) - 1]]
        bs += 1 / x[sort[-1]]

    ks = np.negative(bs)
    temp = ks[:,None] * x
    np.log1p(temp, out=temp)
    np.mean(temp, axis=1, out=ks)

    L = bs / ks
    np.negative(L, out=L)
    np.log(L, out=L)
    L -= ks
    L -= 1
    L *= n

    temp = L - L[:,None]
    np.exp(temp, out=temp)
    w = np.sum(temp, axis=1)
    np.divide(1, w, out=w)

    # remove negligible weights
    dii = w >= 10 * np.finfo(float).eps
    if not np.all(dii):
        w = w[dii]
        bs = bs[dii]
    # normalise w
    w /= w.sum()

    # posterior mean for b
    b = np.sum(bs * w)
    # Estimate for k, note that we return a negative of Zhang and
    # Stephens's k, because it is more common parameterisation.
    temp = (-b) * x
    np.log1p(temp, out=temp)
    k = np.mean(temp)
    if return_quadrature:
        np.negative(x, out=temp)
        temp = bs[:, None] * temp
        np.log1p(temp, out=temp)
        ks = np.mean(temp, axis=1)
    # estimate for sigma
    sigma = -k / b * n / (n - 0)
    # weakly informative prior for k
    a = 10
    k = k * n / (n+a) + a * 0.5 / (n+a)
    if return_quadrature:
        ks *= n / (n+a)
        ks += a * 0.5 / (n+a)

    if return_quadrature:
        #print('K:'+str(k))
        return k, sigma, ks, w
    else:
        #print('K:'+str(k))
        return k, sigma

def gpinv(p, k, sigma):
    """Inverse Generalised Pareto distribution function."""
    x = np.empty(p.shape)
    x.fill(np.nan)
    if sigma <= 0:
        return x
    ok = (p > 0) & (p < 1)
    if np.all(ok):
        if np.abs(k) < np.finfo(float).eps:
            np.negative(p, out=x)
            np.log1p(x, out=x)
            np.negative(x, out=x)
        else:
            np.negative(p, out=x)
            np.log1p(x, out=x)
            x *= -k
            np.expm1(x, out=x)
            x /= k
        x *= sigma
    else:
        if np.abs(k) < np.finfo(float).eps:
            # x[ok] = - np.log1p(-p[ok])
            temp = p[ok]
            np.negative(temp, out=temp)
            np.log1p(temp, out=temp)
            np.negative(temp, out=temp)
            x[ok] = temp
        else:
            # x[ok] = np.expm1(-k * np.log1p(-p[ok])) / k
            temp = p[ok]
            np.negative(temp, out=temp)
            np.log1p(temp, out=temp)
            temp *= -k
            np.expm1(temp, out=temp)
            temp /= k
            x[ok] = temp
        x *= sigma
        x[p == 0] = 0
        if k >= 0:
            x[p == 1] = np.inf
        else:
            x[p == 1] = -sigma / k
    return x

# Evaluate Importance Ratios

In [31]:
Ratio=[]
D=2
N=4
for X in VS:
    P = np.log(np.exp(Joint_Prob(Y,X.detach().numpy(),kernel,N)))
    Q = np.exp(Variational_Dist(X.detach().numpy(),D,kernel))
    Ratio.append(np.divide(P,Q).flatten())


In [33]:
Ratio = np.array(Ratio)
Ratio = Ratio.reshape(-1)
Ratio.shape

(150,)

In [34]:
gpdfitnew(Ratio)

 /home/aneeqr/.local/lib/python3.6/site-packages/ipykernel_launcher.py:186: RuntimeWarning:divide by zero encountered in true_divide
 /home/aneeqr/.local/lib/python3.6/site-packages/ipykernel_launcher.py:187: RuntimeWarning:divide by zero encountered in double_scalars
 /home/aneeqr/.local/lib/python3.6/site-packages/ipykernel_launcher.py:187: RuntimeWarning:invalid value encountered in add
 /home/aneeqr/.local/lib/python3.6/site-packages/ipykernel_launcher.py:207: RuntimeWarning:invalid value encountered in greater_equal
 /home/aneeqr/.local/lib/python3.6/site-packages/ipykernel_launcher.py:227: RuntimeWarning:invalid value encountered in double_scalars


(0.03125, nan)